# Funding Rate Arbitrage Strategy Backtest

This notebook backtests the delta-neutral funding rate arbitrage strategy.

**Strategy Parameters:**
- Min spread: 0.3% hourly
- Leverage: 5x
- Position size: $500 per side
- Exit: Spread < 0.2%, compression > 60%, or max 24h duration
- Stop loss: -3%

**Connectors:** Extended, Lighter, Variational (Perp DEXs)

In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
sys.path.append('/Users/tdl321/quants-lab')
sys.path.append('/Users/tdl321/hummingbot')

from core.backtesting import BacktestingEngine
import datetime
from decimal import Decimal
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Initialize backtesting engine
backtesting = BacktestingEngine(load_cached_data=False)

ModuleNotFoundError: No module named 'motor'

## Configuration

Set up the funding rate arbitrage strategy configuration.

In [ ]:
# Import the strategy configuration
from scripts.v2_funding_rate_arb import FundingRateArbitrageConfig

# Strategy configuration
config = FundingRateArbitrageConfig(
    connectors={"extended_perpetual", "lighter_perpetual", "variational_perpetual"},
    tokens={"KAITO", "MON", "IP", "GRASS", "ZEC", "APT"},
    leverage=5,
    min_funding_rate_profitability=Decimal('0.003'),  # 0.3% hourly
    position_size_quote=Decimal('500'),  # $500 per side
    absolute_min_spread_exit=Decimal('0.002'),  # 0.2%
    compression_exit_threshold=Decimal('0.4'),  # 60% compression
    max_position_duration_hours=24,
    max_loss_per_position_pct=Decimal('0.03'),  # 3% stop loss
    trade_profitability_condition_to_enter=False
)

print("Strategy Configuration:")
print(f"Min Spread: {config.min_funding_rate_profitability:.2%}")
print(f"Leverage: {config.leverage}x")
print(f"Position Size: ${config.position_size_quote} per side")
print(f"Exit Spread: {config.absolute_min_spread_exit:.2%}")
print(f"Max Duration: {config.max_position_duration_hours}h")

## Backtesting Period

Define the time period for backtesting.

In [ ]:
# Backtesting time period
start = int(datetime.datetime(2024, 10, 1).timestamp())
end = int(datetime.datetime(2024, 10, 31).timestamp())
backtesting_resolution = "1h"  # Hourly resolution for funding rate strategy

print(f"Backtest Period: {datetime.datetime.fromtimestamp(start)} to {datetime.datetime.fromtimestamp(end)}")
print(f"Resolution: {backtesting_resolution}")

## Run Backtest

Execute the backtesting engine with the configured strategy.

**Note:** This requires historical funding rate data for Extended, Lighter, and Variational perpetual exchanges.

In [ ]:
# Run backtesting
# Note: This will fail if funding rate data is not available for the configured exchanges
# You may need to collect historical funding rate data first

try:
    backtesting_result = await backtesting.run_backtesting(config, start, end, backtesting_resolution)
    print("✅ Backtest completed successfully!")
except Exception as e:
    print(f"❌ Backtest failed: {e}")
    print("\nNote: You may need to collect historical funding rate data for:")
    print("- Extended Perpetual")
    print("- Lighter Perpetual")
    print("- Variational Perpetual")

## Results Summary

View the high-level performance metrics.

In [ ]:
# Display results summary
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

## Detailed Analysis

### Executors DataFrame

This contains all position executions with entry/exit details.

In [ ]:
# Get executors dataframe
executors_df = backtesting_result.executors_df

print(f"Total Positions: {len(executors_df)}")
print(f"Profitable Positions: {(executors_df['net_pnl_quote'] > 0).sum()}")
print(f"Losing Positions: {(executors_df['net_pnl_quote'] < 0).sum()}")
print(f"Win Rate: {(executors_df['net_pnl_quote'] > 0).mean():.2%}")
print(f"\nAverage Profit: ${executors_df[executors_df['net_pnl_quote'] > 0]['net_pnl_quote'].mean():.2f}")
print(f"Average Loss: ${executors_df[executors_df['net_pnl_quote'] < 0]['net_pnl_quote'].mean():.2f}")

executors_df.head(10)

### PNL per Trade

Scatter plot showing profit/loss for each arbitrage position.

In [ ]:
# Create profitable flag
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# PNL scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Funding Rate Arbitrage Position',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (USD)'},
    hover_data=['filled_amount_quote', 'side', 'trading_pair']
)

fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (USD)",
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',
    paper_bgcolor='rgba(0,0,0,0.8)',
    font=dict(color="white"),
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

fig.add_hline(y=0, line_dash="dash", line_color="lightgray")
fig.show()

### PNL Distribution

Histogram showing the distribution of profits and losses.

In [ ]:
fig = px.histogram(
    executors_df, 
    x='net_pnl_quote', 
    title='PNL Distribution',
    labels={'net_pnl_quote': 'Net PNL (USD)'},
    nbins=50
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0.8)',
    paper_bgcolor='rgba(0,0,0,0.8)',
    font=dict(color="white"),
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

fig.show()

### Cumulative PNL

Track cumulative profit over time.

In [ ]:
# Calculate cumulative PNL
executors_df_sorted = executors_df.sort_values('timestamp')
executors_df_sorted['cumulative_pnl'] = executors_df_sorted['net_pnl_quote'].cumsum()

fig = px.line(
    executors_df_sorted,
    x='timestamp',
    y='cumulative_pnl',
    title='Cumulative PNL Over Time',
    labels={'timestamp': 'Timestamp', 'cumulative_pnl': 'Cumulative PNL (USD)'}
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0.8)',
    paper_bgcolor='rgba(0,0,0,0.8)',
    font=dict(color="white"),
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

fig.show()

### Performance by Token

Breakdown of performance by trading pair.

In [ ]:
# Performance by token
token_performance = executors_df.groupby('trading_pair').agg({
    'net_pnl_quote': ['sum', 'mean', 'count'],
}).round(2)

token_performance.columns = ['Total PNL', 'Avg PNL', 'Count']
token_performance['Win Rate'] = executors_df.groupby('trading_pair')['profitable'].mean()
token_performance = token_performance.sort_values('Total PNL', ascending=False)

print("\nPerformance by Token:")
print(token_performance)

In [ ]:
# Bar chart of total PNL by token
fig = px.bar(
    token_performance.reset_index(),
    x='trading_pair',
    y='Total PNL',
    title='Total PNL by Token',
    labels={'trading_pair': 'Token', 'Total PNL': 'Total PNL (USD)'},
    color='Total PNL',
    color_continuous_scale=['red', 'yellow', 'green']
)

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0.8)',
    paper_bgcolor='rgba(0,0,0,0.8)',
    font=dict(color="white"),
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

fig.show()

### Position Duration Analysis

How long were positions held on average?

In [ ]:
# Calculate position duration (if available in executors_df)
if 'close_timestamp' in executors_df.columns and 'timestamp' in executors_df.columns:
    executors_df['duration_hours'] = (executors_df['close_timestamp'] - executors_df['timestamp']) / 3600
    
    print(f"Average Duration: {executors_df['duration_hours'].mean():.2f} hours")
    print(f"Median Duration: {executors_df['duration_hours'].median():.2f} hours")
    print(f"Max Duration: {executors_df['duration_hours'].max():.2f} hours")
    
    fig = px.histogram(
        executors_df,
        x='duration_hours',
        title='Position Duration Distribution',
        labels={'duration_hours': 'Duration (hours)'},
        nbins=30
    )
    
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0.8)',
        paper_bgcolor='rgba(0,0,0,0.8)',
        font=dict(color="white"),
        xaxis=dict(gridcolor="gray"),
        yaxis=dict(gridcolor="gray")
    )
    
    fig.show()
else:
    print("Duration data not available in executors_df")

### Risk Metrics

Calculate key risk-adjusted performance metrics.

In [ ]:
# Risk metrics
total_pnl = executors_df['net_pnl_quote'].sum()
max_profit = executors_df['net_pnl_quote'].max()
max_loss = executors_df['net_pnl_quote'].min()
sharpe_ratio = executors_df['net_pnl_quote'].mean() / executors_df['net_pnl_quote'].std() if executors_df['net_pnl_quote'].std() > 0 else 0

# Calculate max drawdown
cumulative_pnl = executors_df_sorted['cumulative_pnl']
running_max = cumulative_pnl.expanding().max()
drawdown = cumulative_pnl - running_max
max_drawdown = drawdown.min()

print("\n" + "="*50)
print("RISK METRICS")
print("="*50)
print(f"Total PNL: ${total_pnl:.2f}")
print(f"Max Single Profit: ${max_profit:.2f}")
print(f"Max Single Loss: ${max_loss:.2f}")
print(f"Max Drawdown: ${max_drawdown:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Profit Factor: {abs(executors_df[executors_df['net_pnl_quote'] > 0]['net_pnl_quote'].sum() / executors_df[executors_df['net_pnl_quote'] < 0]['net_pnl_quote'].sum()):.2f}")
print("="*50)

## Parameter Sensitivity Analysis

Test how different parameters affect performance.

**Note:** This is a placeholder for testing different configurations.

In [ ]:
# Example: Test different min spread thresholds
spread_thresholds = [0.002, 0.003, 0.004, 0.005]  # 0.2%, 0.3%, 0.4%, 0.5%

results_by_threshold = []

print("Testing different min spread thresholds...\n")

for threshold in spread_thresholds:
    print(f"Testing {threshold:.2%} min spread...")
    # Create new config with different threshold
    test_config = FundingRateArbitrageConfig(
        connectors={"extended_perpetual", "lighter_perpetual", "variational_perpetual"},
        tokens={"KAITO", "MON", "IP"},
        leverage=5,
        min_funding_rate_profitability=Decimal(str(threshold)),
        position_size_quote=Decimal('500'),
        absolute_min_spread_exit=Decimal('0.002'),
        compression_exit_threshold=Decimal('0.4'),
        max_position_duration_hours=24,
        max_loss_per_position_pct=Decimal('0.03'),
        trade_profitability_condition_to_enter=False
    )
    
    # Run backtest (commented out - uncomment when data is available)
    # test_result = await backtesting.run_backtesting(test_config, start, end, backtesting_resolution)
    # results_by_threshold.append({
    #     'threshold': threshold,
    #     'total_pnl': test_result.executors_df['net_pnl_quote'].sum(),
    #     'num_trades': len(test_result.executors_df),
    #     'win_rate': (test_result.executors_df['net_pnl_quote'] > 0).mean()
    # })

# Uncomment when running actual tests:
# sensitivity_df = pd.DataFrame(results_by_threshold)
# print(sensitivity_df)

## Export Results

Save backtest results for further analysis.

In [ ]:
# Export executors to CSV
output_path = '/Users/tdl321/hummingbot/backtest_results_funding_arb.csv'
executors_df.to_csv(output_path, index=False)
print(f"✅ Results exported to: {output_path}")